# Instructions

# MetAtlas analysis workflow

This notebook is for performing a targeted metabolomics analysis on your raw data files.  In order for this to work, you data files must be uploaded to the nersc raw data directory with appropriate permissions.  A fileconverter runs at nersc to convert as follows:
.raw > .mzml > .h5 > pactolus.gz  This conversion must happen before you can proceed with data analysis.  A Northen Lab staff member can help you transfer and convert your files if they are not already at nersc.
***
/global/project/projectdirs/metatlas/raw_data/SUBFOLDER
***

A targeted analysis requires a list of compounds to search for in your sample files.  Thus, at a minimum, you must have some information on their m/z ratio in order to get started.  The inputs for this notebook, include the location of your raw files, and an atlas listing out these compounds.  

Our lab has analyzed thousands of compounds using standardized LCMS methods.  You may use one of these libraries to create an atlas for your sample analysis, assuming you used one of the same standard LCMS methods.

Analysis steps

<ol>
<li>Create your atlas csv external to jupyter: typically you will either have a custom atlas or generate an atlas from one of our EMA libraries.  These can be found in the shared google drive, under atlases.</li>
<li>Run through with your initial atlas</li>
<li>Use the RT adjuster to mark the quality of the peaks and MSMS matches, and also adjust your RT bounds</li>
<li>Reload your atlas from the "get atlas" block</li>
<li>Export your atlas, filter out the compounds marked under id notes as remove.  Save and upload your new atlas</li>
<li>Repeat the analysis using your filtered atlas.</li>
    <li>On your final export, make sure you are excluding system blanks and QC files in your output.  If you have extraction blanks you should include those.</li>
    </ol>
    
There is a database that will store your atlas and registered run files.  During the analysis process you will pull these as local variables. These are used to pull out EIC and MSMS data which is stored in a variable called metatlas_dataset.  All output files are generated from that.  It is a local variable so is not stored in the db - thus if your kernel dies partway through, you need to rerun through the notebook to regenerate that variable.

IMPORTANT: Anytime you adjust your RTs in the interactive plot, you need to retreive your atlas again before exporting any files.  The changes are stored in the db and the exports are made from locally stored variables/dataframes.

# index
1. Import python packages
2. Create groups
3. Select groups
4. Create Atlases
5. Select Atlas
6. Annotate data
7. Correct RT bounds
8. Export results
9. Tools

# Helpful tips

1. When entering search strings, use % for wildcard: %peas will return "positive_peas" and "positive_greenpeas", but not "positive_greenpeas_HILIC", for that use %peas%, you can also put multiple strings via %pos%peas%
2. Set your kernel to "Metatlas Targeted".  If not sure how, check w Ben or Will.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# full name of input atlas to load from database or name to give atlas loaded from CSV file
source_atlas = None

# full path to atlas CSV file or None if loading an atlas from the database
csv_atlas_file_name = None

# one of 'positive' or 'negative'
polarity = "positive"

# one of 'data_QC', 'ISTDsEtc', 'FinalEMA-HILIC', or 'other'
output_type = "FinalEMA-HILIC"

# an integer, increment if you need to redo your analysis
# will be appended to your username to create analysis_id
analysis_number = 0

# experiment ID that must match the parent folder containing the LCMS output files
# An example experiment ID is '20201116_JGI-AK_LH_506489_SoilWarm_final_QE-HF_HILICZ_USHXG01530'
experiment = "REPLACE ME"

# A list of experiment IDs to pull LCMS runs from
# if None, then the value of the "experiment" variable will be used
run_batches = None

# Exclude files with names containing any of the substrings in this list. Eg., ['peas', 'beans']
exclude_files = []

# Exclude groups with names containing any of the substrings in this list.
# 'POS' or 'NEG' will be auto-appended later, so you shouldn't use them here.
exclude_groups = ["QC", "InjBl"]

# thresholds for filtering out compounds with weak MS1 signals
num_points = 6
peak_height = 4e5

# if loading an atlas from CSV, assign all entries this mz_tolerance (in ppm)
mz_tolerance = 5

# file name of the file containing the reference MSMS data
msms_refs_file_name = '/global/project/projectdirs/metatlas/projects/spectral_libraries/msms_refs_v3.tab'

# list of substrings that will group together when creating groups
# this provides additional grouping beyond the default grouping on field #12
groups_controlled_vocab = ["QC", "InjBl", "ISTD"]

# list of tuples contain string with color name and substring pattern.
# Lines in the EIC plot will be colored by the first substring pattern
# that has a match within the name of the hdf5_file. The order they are
# listed in your list is the order they are displayed in the overlays
# (first is front, last is back). Named colors available in matplotlib
# are here: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
# or use hexadecimal values '#000000'. Lines default to black.
line_colors = [('green','ISTD'), ('red','NoD2O')]

# Setting this to True will remove the cache of MSMS hits
# if you don't see MSMS data for any of your compounds in RT adjuster GUI,
# then you might want to try settings this to True. However, it will
# make your notebook take significantly longer to run.
# The cache is per experiment, so clearing the cache will impact other
# notebooks for this same experiment.
clear_cache = False

# subfolder to be created within project_directly to hold data from this analysis
output_subfolder='Anuvia/Pilot4_AgriSoils_pooled/HILICZ'

# The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Full path to the directory where you want this notebook to store data.
# A subdirectory will be auto created within this directory for each project.
# You can place this anywhere on cori's filesystem, but placing it within your
# global home directory is recommended so that you do not need to worry about
# your data being purged. Each project will take on the order of 100 MB.
project_directory='/global/homes/k/kblouie/metabolomics/Plots/'

# ID from Google Drive URL for base output folder .
# The default value is the ID that corresponds to 'JGI_Metabolomics_Projects'.
google_folder = "0B-ZDcHbPi-aqZzE5V3hOZFc0dms"

# maximum number of CPUs to use
# when running on jupyter.nersc.gov, you are not allowed to set this above 4
max_cpus = 4

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=doejgi/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
try:
    with Path("/metatlas_image_version").open(mode="r", encoding="utf-8") as f:
        version = f.readlines()[0].rstrip()
except FileNotFoundError:
    version = "0"
try:
    has_root_kernel = Path('/root/.local/share/jupyter/kernels/papermill/kernel.json').is_file()
except PermissionError:
    has_root_kernel = False
if not (version == "1" and (kernel_file_name.is_file() or has_root_kernel)):
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical("CRITICAL: Notebook kernel has been updated. Restart kernel and re-run notebook.")
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Please check that the kernel is set to "Metatlas Targeted".')
    raise StopExecution from err
notebook.setup(log_level, source_code_version_id)

import getpass
import itertools
import multiprocessing as mp
import os
import shutil
import time
import pandas as pd
from IPython.display import display, HTML
from metatlas.tools import fastanalysis as fa
from metatlas.plots import dill2plots as dp
from metatlas.io import metatlas_get_data_helper_fun as ma_data
from metatlas.datastructures import metatlas_objects as metob
from metatlas.datastructures.analysis_identifiers import AnalysisIdentifiers
from metatlas.datastructures import metatlas_dataset as mads

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
assert source_atlas is not None
run_batches = run_batches if run_batches is not None else [experiment] 
%matplotlib widget
username = getpass.getuser()

# Set atlas, project and output directories from your nersc home directory

STEP 1
1. Create a project folder name for this analysis by replacing the PROJECTDIRECTORY string text in red below.  Make sure to update the rest of the direcory to point to your home directory.  The pwd block will print out the directory where this jupyter notebook is stored.
2. Create a subdirectory name for the output, each run through you may want to create a new output folder.
3. When you run the block the folders will be created in your home directory.  If the directory already exists, the block will just set the path for use with future code blocks.

STEP 2
1.  Enter the nersc path where you have stored your atlas files using one of the two optional lines below:
    1. Set a custom path
    2. Use the project and output subdirectory from above

In [ ]:
pwd

In [ ]:
#STEP 1
output_dir = os.path.join(project_directory,output_subfolder)
os.makedirs(project_directory, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 2. Create Groups (named variables that hold your replicates of each sample)

### You must assign your raw files into experimental groups for analysis.  These are used for downstream statistics and for selection of specific groups for filtering to subsets of files for analysis (Ex. just pos or just neg).

The groups are created from common file headers and the unique group names. The convention our lab group uses for filenames is as follows: 
***
DATE_NORTHENLABINITIALS_COLLABINITIALS_PROJ_EXP_SAMPSET_SYSTEM_COLUMN-method_SERIAL_POL_ACQ_SAMPLENUMBER_ SAMPLEGROUP_REP_OPTIONAL_SEQ 

Ex.:20180105_SK_AD_ENIGMA_PseudoInt_R2ADec2017_QE119_50454_123456_POS_MSMS_001_Psyringae-R2A-30C-20hr_Rep01_NA_Seq001.raw
***
The common header consists of the fields 0-10: DATE_NORTHENLABINITIALS_COLLABINITIALS_PROJ_EXP_SAMPSET_SYSTEM_COLUMN-method_SERIAL_POL_ACQ 

The sample group name is commonly field # 12 (between underscore 11 and 12) -0 indexed-


# Find your files
1. On the first line of the block below, set the 'experiment' and 'name' variables to find your files.  These fields require wildcards for partial string searches
2. 'Experiment' is the folder name within global/project/projectdirs/metatlas/raw_data, that will be emailed to you when the files are uploaded to NERSC.  You can also look in the raw_data directory for the NERSC user who uploaded your files; your experiment folder should be in there.
3. 'name' is string that will match a subset of your files within that folder.  

In [ ]:
files = dp.get_metatlas_files(experiment=[f"{b}%" for b in run_batches], name="%", most_recent=True)
df = metob.to_dataframe(files)
df[['experiment','name','username','acquisition_time']]

In [ ]:
len(files)

# OPTION A: Automated Group Maker

This will attempt to create groups in an automated fashion (rather than filling out a spreadsheet with a list of files and group names).  If your files are all in one folder at nersc, you can use this options.  If not, use option B below.

A long group name consisting of the common header + either controlled vocab value or field #12 along with a short group name (just controlled vocab or field #12) will be stored in a local variable.  The short group names can be used on plots.


1. STEP 1: View the groups
    1. Pick an experiment folder to look for files in on the metob.retrieve function
    2. Enter controlled vocabulary for control files to put select files into groups when control string may be in a different field (not #12) or as a randomly placed substring within a field (ex. if 'InjBl' is included in your controlled vocab list, files like _InjBl-MeOH_ and _StartInjBl_ will group together)
    3. If your group name is not between _ 11 and 12 you can adjust those values in the split commands below.  All other (non-controlledvocab) groups will be created from that field.
2. STEP 2: Create the groups variable after checking the output from STEP 1
3. STEP 3: <br />
    Option A: If everything looks fine the group names and short names, Store groups once you know you have files in correct groups by running and checking the output of STEPS 1 and 2.<br />
    Option B (optional): If you would like to edit the groups, uncomment the options B-I and B-II. Run Option B-I to export a prefilled tab infosheet. Edit the file and then run Option B-II to import the new groups and save it. 

In [ ]:
#STEP 1: View the groups

files = dp.get_metatlas_files(experiment=[f"{b}%" for b in run_batches], name="%", most_recent=True)
    
controlled_vocab = ['QC','InjBl','ISTD'] #add _ to beginning. It will be stripped if at begining
version_identifier = f"{username}{analysis_number}"
file_dict = {}
groups_dict = {}
for f in files:
    k = f.name.split('.')[0]
    #     get index if any controlled vocab in filename
    indices = [i for i, s in enumerate(controlled_vocab) if s.lower() in k.lower()]
    prefix = '_'.join(k.split('_')[:11])
    if len(indices)>0:
        short_name = controlled_vocab[indices[0]].lstrip('_')
        group_name = '%s_%s_%s'%(prefix,version_identifier,short_name)
        short_name = k.split('_')[9]+'_'+short_name # Prepending POL to short_name
    else:
        short_name = k.split('_')[12]
        group_name = '%s_%s_%s'%(prefix,version_identifier,short_name)
        short_name = k.split('_')[9]+'_'+k.split('_')[12]  # Prepending POL to short_name
    file_dict[k] = {'file':f,'group':group_name,'short_name':short_name}
    groups_dict[group_name] = {'items':[],'name':group_name,'short_name':short_name}
df = pd.DataFrame(file_dict).T
df.index.name = 'filename'
df.reset_index(inplace=True)#['group'].unique()
df.drop(columns=['file'],inplace=True)
for ug in groups_dict.keys():
    for file_key,file_value in file_dict.items():
        if file_value['group'] == ug:
            groups_dict[ug]['items'].append(file_value['file'])
df.head(100)

In [ ]:
#STEP 2: create the groups variable, if the above looks OK

groups = []
for group_key,group_values in groups_dict.items():
    g = metob.Group(name=group_key,items=group_values['items'],short_name=group_values['short_name'])
    groups.append(g)        
    for item in g.items:
        print(g.name,g.short_name,item.name)
    print('')

In [ ]:
# STEP 3 Option A: store the groups variable content in the DB (currently only the long group name is stored)
metob.store(groups)

## Make data frame of short filenames and samplenames
Uncomment the below 2 blocks to make short file names and smaple names.<br>
This creates a dataframe and a csv file which can be edited, exported and imported. 

In [ ]:
# Make short_filename and short_samplename 
files = dp.get_metatlas_files(experiment=[f"{b}%" for b in run_batches], name="%", most_recent=True)

short_filename_delim_ids = [0,2,4,5,7,9,14]
short_samplename_delim_ids = [9,12,13,14]
short_names_df = pd.DataFrame(columns=['sample_treatment','short_filename','short_samplename'])
ctr = 0
for f in files:
    short_filename = []
    short_samplename = []
    tokens = f.name.split('.')[0].split('_')
    for id in short_filename_delim_ids:
        short_filename.append(str(tokens[id]))
    for id in short_samplename_delim_ids:
        short_samplename.append(str(tokens[id]))
    short_filename = "_".join(short_filename)
    short_samplename = "_".join(short_samplename)
    short_names_df.loc[ctr, 'full_filename'] = f.name.split('.')[0]
    short_names_df.loc[ctr, 'sample_treatment'] = str(tokens[12]) # delim 12
    short_names_df.loc[ctr, 'short_filename'] = short_filename
    short_names_df.loc[ctr, 'short_samplename'] = short_samplename
    short_names_df.loc[ctr, 'last_modified'] = pd.to_datetime(f.last_modified,unit='s')
    ctr +=1
short_names_df.sort_values(by='last_modified', inplace=True)
short_names_df.drop(columns=['last_modified'], inplace=True)
short_names_df.drop_duplicates(subset=['full_filename'], keep='last', inplace=True)
short_names_df.set_index('full_filename', inplace=True)
short_names_df.to_csv(os.path.join(output_dir, 'short_names.csv'), sep=',', index=True)

# 3. Select groups of files to operate on

Here, you will assign your database groups to a local variable which will be used downstream in the notebook for analyzing your data with an atlas.

1. in block below, fill out the fields for name, include_list and exclude_list using text strings from the group names you created in the previous step.  The include/exlcude lists do not need wildcards.  Name is a string unique to all of your groups (ex. fields 0-11 of your filenames)

### Typically, you will run one polarity at a time.

In [ ]:
groups = dp.select_groups_for_analysis(name = f"{experiment}%",    # <- edit text search string here
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG','QC','InjBL','InjBl'])# ex. ['QC','Blank'])
print("sorted groups")
groups = sorted(groups, key=lambda x: x.name)
for i,a in enumerate(groups):
    print(i, a.name)

In [ ]:
# to view metadata about your groups, run the block below
metob.to_dataframe(groups)

# 4. Create new Atlas entries in the Metatlas database from a csv file

## QC, IS, and EMA template atlases are available on the google drive.

1. Create your atlas as a csv file, check that it looks correct (has all the correct headers and no blank values in rows; all columns are the correct data type
2. Save it with the type of atlas (EMA, QC or IS), your initials, the experiment name, the polarity, and the version or timestamp
3. Upload it to your nersc project directory (the one you named above).  (If it doesn't work, double check your file permissions are set to at least rw-rw----).
4. Run blocks below to create the DB entries for negative and positive mode atlases
5. WARNING: Don't run this block over and over again - it will create multiple new DB entries with the same atlas name

Required Atlas file headers:

rt_min,rt_max,rt_peak,mz,mz_tolerance,adduct,polarity,identification_notes

values in rows must be completed for all fields except inchi_key (leaving this blank will not allow you to perform MSMS matching below), and identification notes

INFO: store=True will register your atlas in the database.  If you are not sure if your atlas structure is correct, set store=False for the first time your run the block to check if you get an error.  If there is no error, then rerun it with store=True.

# 5. Select Atlas to use
## 5a. Load an existing atlas from the database
1. The first block will retrieve a list of atlases matching the 'name' string that you enter.  Also, you must enter your username.
2. The next block will select one from the list, using the index number.  Make sure to enter the index number for the atlas you want to use for your analysis by setting in this line: my_atlas = atlases[0]

In [ ]:
if csv_atlas_file_name is None:
    atlases = metob.retrieve('Atlas',name=source_atlas, username=username)
    names = []
    for i,a in enumerate(atlases):
        print(i,a.name,pd.to_datetime(a.last_modified,unit='s'))#len(a.compound_identifications)

In [ ]:
if csv_atlas_file_name is None:
    my_atlas = atlases[0]
    atlas_df = ma_data.make_atlas_df(my_atlas)
    atlas_df['label'] = [cid.name for cid in my_atlas.compound_identifications]
    print(my_atlas.name)
    metob.to_dataframe([my_atlas])
    # the first line of the output will show the dimensions of the atlas dataframe

### 5b. Load atlas from CSV file

In [ ]:
if csv_atlas_file_name is not None:
    my_atlas = dp.make_atlas_from_spreadsheet(csv_atlas_file_name, 
                                              source_atlas,
                                              filetype='csv',
                                              sheetname='',
                                              polarity = polarity,
                                              store=True,
                                              mz_tolerance = mz_tolerance)

In [ ]:
# OPTIONAL: to view your atlas, run this block
print(my_atlas.name)
atlas_df

# 6. Get EICs and MSMS for all files in your groups, using all compounds in your atlas.

### This block builds the metatlas_dataset variable.  This holds your EIC data (mz, rt, intensity values within your mz and rt ranges).

The EIC data contains mz, intensity and RT values across your RT range.  There are two parameters that you will need to edit: extra_time and extra_mz.  Extra time will collect mz, intensity and RT values from outside of your atlas defined min and max rt values.  For example if your rt_min = 1.0, and rt_max = 2.0 and you set extra_time to 0.3, then your new rt range will be 0.7 to 2.3.  This is helpful for checking if you have nearby peaks at the same m/z.  Extra_mz should only be used for troubleshooting.  You should keep this at 0 unless you believe you have poor mass accuracy during your run.  Other ways to address this issue is by changing the mz_tolerance values in your atlas.  Before changing this value, you should check in with a metatlas experienced lab member to discuss when/how to use this value.

1. Change the value in "extra_time = 0.0" to something like 0.5 to 1.0 for the first EMA runthrough on your files.  This will take longer but collect msms outside your retention windows which allows you to check the msms of nearby peaks before adjusting your rt bounds around the correct peak.
2. extra_mz should almost always be set to 0.0   If you need to troubleshoot a low mz compound you could potentially use this value to run it back through with a larger mz error window than what was specified in your atlas (ppm tolerance).

>On Your final runthrough, set extra_time to 0

In [ ]:
all_files = []
for my_group in groups:
    for my_file in my_group.items:
        extra_time = 0.5         # NOTE: 0.75 for the first run, 0.5 for final 
        extra_mz = 0.00
        all_files.append((my_file,my_group,atlas_df,my_atlas,extra_time,extra_mz))
pool = mp.Pool(processes=min(4, len(all_files)))
t0 = time.time()
metatlas_dataset = pool.map(ma_data.get_data_for_atlas_df_and_file, all_files)
pool.close()
pool.terminate()
print(time.time() - t0)

In [ ]:
# Make data sources tables (atlas_metadata.tab, groups_metadata.tab, groups.tab and [atlasname]_originalatlas.tab within data_sources subfolder)
#ma_data.make_data_sources_tables(groups, my_atlas, output_dir) 

ma_data.make_data_sources_tables(groups, my_atlas, output_dir, polarity=polarity[:3].upper())

# 6b Optional: Filter atlas for compounds with no or low signals

Uncomment the below 3 blocks to filter the atlas.
Please ensure that correct polarity is used for the atlases.

In [ ]:
atlas_df_passing = dp.filter_atlas(atlas_df, metatlas_dataset, num_points, peak_height)
print("# Compounds in Atlas: "+str(len(atlas_df)))
print("# Compounds passing filter: "+str(len(atlas_df_passing)))

## Create new atlas and store in database
This block creates a filtered atlas with a new name !!
Automatically selects this atlas for processing. 
Make sure to use this atlas for downstream analyses. (NOTE: If you restart kernel or come back to the analysis, you need to reselect this newly created filtered atlas for processing)

In [ ]:
atlas_passing = my_atlas.name+'_filteredby-datapnts'+str(num_points)+'-pkht'+str(peak_height)
myAtlas_passing = dp.make_atlas_from_spreadsheet(atlas_df_passing,
                          atlas_passing,
                          filetype='dataframe',
                          sheetname='',
                          polarity = polarity,
                          store=True,
                          mz_tolerance = 12)

atlases = dp.get_metatlas_atlas(name=atlas_passing,do_print = True, most_recent=True)

In [ ]:
my_atlas=atlases[0]

## Get MSMS Hits - Skip this if your next step is the RT adjuster GUI.
### One of the two blocks below builds the hits variable.  This holds your MSMS spectra (from within your mz, and rt ranges, and within the extra time indicated above).

Skip this step if you are about to use the RT adjuster GUI as it will redo the calculation.

There are two options for generating the hits variable:
1. block A: use when your files have msms. It create the hits variable and also saves a binary (pickled) serialized hits file to the output directory.
2. block B: only run if your files were collected in MS1 mode
3. If you have already run block A and then the kernel dies, you can skip block A and directly unplickle the binary hits file from the output directory. Skip block A, uncomment the Optional block and run it. 

In [ ]:
##BLOCK A
t0 = time.time()

hits=dp.get_msms_hits(metatlas_dataset,extra_time=True,keep_nonmatches=True, frag_mz_tolerance=0.01, ref_loc=msms_refs_file_name)
#pickle.dump(hits, open(os.path.join(output_dir,short_polarity+'_hits.pkl'), "wb"))

print(time.time() - t0)
print('%s%s' % (len(hits),' <- total number of MSMS spectra found in your files'))

# 7. Adjust Retention Times. 

This block creates an interactive plot.  The top panel displays MSMS from within the two green RT bounds selected below (rt min and max, initially set in atlas).  When the database holds reference spectra, mirror plots are generated with the reference spectra inverted below the sample spectra.  The lower panel displays the EICs overlayed for all of the files in your selected groups.  You can highlight your groups different colors.  It is recommended that you do this, at least, for your extraction blank (or if not available, use a solvent injection blank).  This plot also displays radio buttons that can be interactively selected; the values will be exported in your final identifications table and in your atlas export.  Use these to mark peak/MSMS quality.

How to use:
1. STEP 1: Set peak flag radio buttons
    1. OPTION A (custom flags): fill out the peak flags list (list of strings) 
        peak_flag_list = ('A','B') some recommendations are below.  
    2. OPTION B (default flags): comment out the custom peak_flag_list line.  Uncomment the default peak_flags = "". 
        Flags default to: keep, remove, unresolvable isomers, check.
2. STEP 2: Set EIC colors
    1. Option A (custom EIC colors): fill out the colorlist in the format of below
   > ***
   > colorlist = [['color1nameorhexadec','partialgroupstring1'],
   >         ['color2nameorhexadec','partialgroupstring2']]
   > ***
    <ul><li>You can add more comma delimited colors/groups as needed.</li>
    <li>These are partial strings that match to you file names (not your group names).</li>
    <li>The order they are listed in your list is the order they are displayed in the overlays (first is front, last is back)</li>
    <li>Named colors available in matplotlib are here: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
          or use hexadecimal values '#000000'</li></ul>
    B. Option B (default EIC colors): comment out the custom colorlist lines and uncomment the default colorlist = "". 
        Colors all default to black.
3. User the right/left buttons on your keyboard to cycle through compounds in your atlas.
4. Use the up/down buttons on your keyboard to cycle through MSMS spectra within the RT bounds of the lower plot.
5. Use the horizontal rt min and rt max bars below the plots to adjust the rt bounds around your peak.  If there are multiple peaks, select one at a time and then click up/down to update the msms available in that new RT range.  If necessary evaluate your data in an external program such as mzmine to make sure you are selecting the correct peak.

TIPS: use compound_idx = 0 in step 3 to change to a different compound in your atlas using the index number.  If your plot does not fit in your browser window, adjust height and width values.  Use alpha to change the transparency of the lines this is a value 0 (transparent) to 1 (opaque).

DO NOT change your RT theoretical peak (the purple line).  It is locked from editing (unless you change a hidden parameter) and only to be changed in special cases.  The measured retention times of your peaks will be calculated and exported in your output files.  These will be compared with the RT theoreticals and used in your evidence of identification table.

In [ ]:
ids = AnalysisIdentifiers(project_directory, experiment, output_type, polarity, analysis_number, google_folder,
          source_atlas=my_atlas.name,
          copy_atlas=False,
          username=username,
          exclude_files=None,
          include_groups=None,
          exclude_groups=None,
          groups_controlled_vocab=None,
          lcmsruns=files,
          all_groups=groups)
if clear_cache:
    shutil.rmtree(ids.cache_dir)
metatlas_dataset = mads.MetatlasDataset(ids=ids, msms_refs_loc=msms_refs_file_name, max_cpus=max_cpus)

In [ ]:
agui = metatlas_dataset.annotation_gui(compound_idx=0, width=15, height=3, colors=line_colors, adjustable_rt_peak=False)

# Comprehensive output generation and RClone upload
The next block generates all outputs and uploads then to Google Drive. If you use the next block, you can skip blocks 8 and beyond.

In [ ]:
mads.post_annotation(metatlas_dataset, require_all_evaluated=False)

## 8. Create filtered atlas excluding compounds marked removed

Re-run the following before filtering atlas
1. Get Groups (include InjBl)
2. Get Atlas
3. Get Data
4. Get MSMS Hits

In [ ]:
#(atlas_all, atlas_kept, atlas_removed) = dp.filter_by_remove(atlas_df, metatlas_dataset)
#print("# Compounds Total: "+str(len(atlas_all)))
#print("# Compounds Kept: "+str(len(atlas_kept)))
#print("# Compounds Removed: "+str(len(atlas_removed)))

(atlas_kept, atlas_removed) = dp.filter_by_remove(atlas_df, metatlas_dataset)
print("# Compounds Kept: "+str(len(atlas_kept)))
print("# Compounds Removed: "+str(len(atlas_removed)))

In [ ]:
#MAKE SURE YOU PUT IN THE CORRECT POLARITY!!!!

atlasfilename=my_atlas.name+'_kept'  # <- enter the name of the atlas to be stored

names = dp.make_atlas_from_spreadsheet(atlas_kept, 
                                       atlasfilename,  # <- DO NOT EDIT THIS LINE
                                       filetype='dataframe',
                                       sheetname='',
                                       polarity = polarity,
                                       store=True,
                                       mz_tolerance = 7
                                      )   

## Re-run the following after filtering atlas
1. Restart kernel
2. Get Groups
3. Get Atlas (look for the *_kept atlas)
4. Get Data
5. Get MSMS Hits

# 9. Export results files

### Export Atlas to a Spreadsheet

The peak flags that you set and selected from the rt adjuster radio buttons will be saved in a column called id_notes

In [ ]:
atlas_identifications = dp.export_atlas_to_spreadsheet(my_atlas,os.path.join(output_dir,'%s_%s%s.csv' % (polarity,my_atlas.name,"export")))
print(my_atlas.name)

### Export MSMS match scores, stats sheets, and final identification table

This block creates a number of files:

1. compound_scores.csv
2. stats_table.tab
3. filtered and unfiltered peak heights, areas, msms scores, mz centroid, mz ppm error, num of fragment matches, rt delta, rt peak
4. final identification sheet that is formatted for use as a supplemental table for manuscript submission.  You will need to manually complete some columns.  Please discuss with Ben, Katherine, Daniel or Suzie before using for the first time.

THe kwargs below will set the filtering points for the parameters indicated.

In [ ]:
kwargs = {'min_intensity': 1e4,   # strict = 1e5, loose = 1e3
          'rt_tolerance': .5,    #>= shift of median RT across all files for given compound to reference
          'mz_tolerance': 20,      # strict = 5, loose = 25; >= ppm of median mz across all files for given compound relative to reference
          'min_msms_score': .6, 'allow_no_msms': True,     # strict = 0.6, loose = 0.3 <= highest compound dot-product score across all files for given compound relative to reference
          'min_num_frag_matches': 1, 'min_relative_frag_intensity': .001}   # strict = 3 and 0.1, loose = 1, 0.01 number of matching mzs when calculating max_msms_score and ratio of second highest to first highest intensity of matching sample mzs
scores_df = fa.make_scores_df(metatlas_dataset,hits)
scores_df['passing'] = fa.test_scores_df(scores_df, **kwargs)

pass_atlas_df, fail_atlas_df, pass_dataset, fail_dataset = fa.filter_atlas_and_dataset(scores_df, atlas_df, metatlas_dataset, column='passing')

fa.make_stats_table(input_dataset = metatlas_dataset, msms_hits = hits, output_loc = output_dir,min_peak_height=1e5,use_labels=True,min_msms_score=0.01,min_num_frag_matches=1,include_lcmsruns = [],exclude_lcmsruns = ['QC'], polarity=polarity)
os.makedirs(os.path.join(output_dir,'stats_tables'), exist_ok=True)
scores_df.to_csv(os.path.join(output_dir,'stats_tables',polarity+'_compound_scores.csv'))

### Export EIC chromatograms as individual pdfs for each compound

1.  There are three options for formatting your EIC output using the "group =" line below:
    1. 'page' will print each sample group on a new page of a pdf file
    2. 'index' will label each group with a letter
    3. None will print all of the groups on one page with very small subplot labels
2. The Y axis scale can be shared across all files using share_y = True or set to the max within each file using share_y = False
3. To use short names for plots, short_names_df should be provided as input. Additionally the header column to be used for short names should be provided as follows (short_names_df=short_names_df, short_names_header='short_samplename'). Header options are sample_treatment, short_filename, short_samplename. These are optional parameters

In [ ]:
group = 'index' # 'page' or 'index' or None
save = True
share_y = True

dp.make_chromatograms(input_dataset=metatlas_dataset, include_lcmsruns = [],exclude_lcmsruns = ['InjBl','InjBL','QC','Blank','blank'], group=group, share_y=share_y, save=save, output_loc=output_dir, short_names_df=short_names_df, short_names_header='short_samplename', polarity=polarity)

### Export MSMS mirror plots as individual pdfs for each compound

1. use_labels = True will use the compound names you provided in your atlas, if you set it to false, the compounds will be named with the first synonym available from pubchem which could be a common name, iupac name, cas number, vendor part number, etc. 
2.  The include and exclude lists will match partial strings in filenames, do not use wildcards.
3. If short_names_df is provided as input, short_samplename is used for plots.

In [ ]:
dp.make_identification_figure_v2(input_dataset = metatlas_dataset, msms_hits=hits, use_labels=True, include_lcmsruns = [],exclude_lcmsruns = ['InjBl','InjBL','QC','Blank','blank','ExCtrl'], output_loc=output_dir,  short_names_df=short_names_df, polarity=polarity)

### Data Sheets
1. To include short names in the output, short_names_df should be provided as input to make_output_dataframe. 
2. ylabel is optional

In [ ]:
peak_height = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_height', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)
peak_area = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_area', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)
mz_peak = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_peak', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)
rt_peak = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [],fieldname='rt_peak', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)
mz_centroid = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_centroid', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)
rt_centroid = dp.make_output_dataframe(input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='rt_centroid', output_loc=os.path.join(output_dir,polarity+'_data_sheets'), short_names_df=short_names_df, polarity=polarity, use_labels=True)

### Box plots

In [ ]:
dp.make_boxplot_plots(rt_peak, output_loc=os.path.join(output_dir, polarity+'_boxplot_rt_peak'), ylabel="RT Peak")
dp.make_boxplot_plots(peak_height, output_loc=os.path.join(output_dir, polarity+'_boxplot_peak_height'), ylabel="Peak Height")
dp.make_boxplot_plots(mz_centroid, output_loc=os.path.join(output_dir, polarity+'_boxplot_mz_centroid'), ylabel="MZ Centroid")